In [1]:
import pandas as pd
import glob
import os
from pathlib import Path

In [2]:
def merge_same_index(df):
    df = df.copy()
    df.index = df.index.str.replace(r"_\d+$", "", regex=True)

    new_rows = {}
    for idx_name, group in df.groupby(df.index):
        if len(group) == 1:
            new_rows[idx_name] = group.iloc[0]
            continue
        
        else:
            conflict_flag = False
            combined = pd.DataFrame()

        conflict = False
        for col in group.columns:
            vals = group[col]
            if isinstance(vals, pd.DataFrame):
                vals = vals.iloc[:, 0]  # 取第一列成 Series
            vals = vals.dropna().unique()
            combined[col] = [vals]  # 存成 list 方便檢查
            
            if len(vals) > 1:
                conflict = True
                break

        if conflict:
            print(f"⚠️ 合併衝突：'{idx_name}' 在同一季出現不同數值，來源列：{list(group.index)}，保留第一個")
            new_rows[idx_name] = group.iloc[0]
        else:
            merged_series = group.apply(
                lambda col: col.dropna().iloc[0] if col.dropna().size > 0 else pd.NA, axis=0
            )
            new_rows[idx_name] = merged_series

    result = pd.DataFrame(new_rows).T
    result = result.reindex(columns=df.columns)
    return result

In [3]:
# 最上層資料夾
base_path = Path.cwd() / "goodinfo" / "my_use"

# 找出所有公司代號資料夾
company_folders = [f for f in glob.glob(os.path.join(base_path, "*")) if os.path.isdir(f)]

for company in company_folders:
    company_name = os.path.basename(company)
    output_file = os.path.join(company, f"{company_name}.xlsx")
    sheets_dict = {}
    missing_info = []

    for subfolder in ["balance_sheet", "cash_flow", "income"]:
        folder_path = os.path.join(company, subfolder)
        if not os.path.exists(folder_path):
            missing_info.append(f"子資料夾缺失: {subfolder}")
            continue

        all_files = glob.glob(os.path.join(folder_path, "*.xls"))
        if not all_files:
            missing_info.append(f"{subfolder} 無 .xls 檔")
            continue

        dfs = []
        
        for file in all_files:
            try:
                tables = pd.read_html(file)
                df = tables[0]
                # 🔴 將第一欄設為 index，但允許重複
                df_index = df.iloc[:, 0].astype(str)  # 原始財務項目名稱
                df_data = df.iloc[:, 1:]  # 其餘欄位
                # 🔴 生成唯一 index，避免 concat 報錯
                df_data.index = [f"{name}_{i}" for i, name in enumerate(df_index)]
                df = df_data
                

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = [f"{a}_{b}" if b != "" else str(a) for a, b in df.columns]

                dfs.append(df)
            # 🔴 捕捉讀取失敗，記錄檔名與錯誤**
            except Exception as e:
                missing_info.append(f"讀取失敗: {os.path.basename(file)} -> {e}")


            if dfs:
                merged_df = pd.concat(dfs, axis=1)
                
                # 🔴 在 concat 後才做「合併同名 index」的邏輯
                merged_df = merge_same_index(merged_df)
                
                #🔴 依季度排序欄位（從新到舊）
                # 假設欄位名稱像 '2007Q4_金額', '2007Q4_％' 這樣
                def quarter_key(col_name):
                    parts = col_name.split('_')
                    q = parts[0]  # '2007Q4'
                    try:
                        year = int(q[:4])
                        quarter = int(q[5])
                        return (-year, -quarter)
                    except:
                        return (0, 0)  # 若不是季度欄位

                
                merged_df = merged_df[sorted(merged_df.columns, key=quarter_key)]
                # 🔴 將合併後的 sheet 存入 sheets_dict**
                sheets_dict[subfolder] = merged_df

    #🔴 只在 sheets_dict 有資料時才生成 Excel**
    if sheets_dict:
        with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
            for sheet_name, df_sheet in sheets_dict.items():
                df_sheet.to_excel(writer, sheet_name=sheet_name, index=True)
        print(f"✅ 已生成 Excel: {output_file}")
        # 🔴 如果有缺失資訊，列印警告**
        if missing_info:
            print(f"⚠️ {company_name} 部分問題: {', '.join(missing_info)}")
    else:
        print(f"❌ {company_name} 沒有有效資料，Excel 未生成")
        # 🔴 列印問題列表**
        if missing_info:
            print(f"⚠️ 問題列表: {', '.join(missing_info)}")

⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來源列：['股東權益', '股東權益']，保留第一個
⚠️ 合併衝突：'負債' 在同一季出現不同數值，來源列：['負債', '負債']，保留第一個
⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來源列：['股東權益', '股東權益', '股東權益', '股東權益']，保留第一個
⚠️ 合併衝突：'負債' 在同一季出現不同數值，來源列：['負債', '負債', '負債', '負債']，保留第一個
⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來源列：['股東權益', '股東權益', '股東權益', '股東權益', '股東權益']，保留第一個
⚠️ 合併衝突：'負債' 在同一季出現不同數值，來源列：['負債', '負債', '負債', '負債']，保留第一個
⚠️ 合併衝突：'業外損益' 在同一季出現不同數值，來源列：['業外損益', '業外損益']，保留第一個
⚠️ 合併衝突：'淨損益' 在同一季出現不同數值，來源列：['淨損益', '淨損益']，保留第一個
⚠️ 合併衝突：'業外損益' 在同一季出現不同數值，來源列：['業外損益', '業外損益']，保留第一個
⚠️ 合併衝突：'淨損益' 在同一季出現不同數值，來源列：['淨損益', '淨損益']，保留第一個
⚠️ 合併衝突：'業外損益' 在同一季出現不同數值，來源列：['業外損益', '業外損益']，保留第一個
⚠️ 合併衝突：'淨損益' 在同一季出現不同數值，來源列：['淨損益', '淨損益']，保留第一個
✅ 已生成 Excel: /home/jovyan/business-cycle/goodinfo/my_use/3036/3036.xlsx
⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來源列：['股東權益', '股東權益']，保留第一個
⚠️ 合併衝突：'負債' 在同一季出現不同數值，來源列：['負債', '負債']，保留第一個
⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來源列：['股東權益', '股東權益', '股東權益', '股東權益']，保留第一個
⚠️ 合併衝突：'負債' 在同一季出現不同數值，來源列：['負債', '負債', '負債', '負債']，保留第一個
⚠️ 合併衝突：'股東權益' 在同一季出現不同數值，來